## Partie I Webscrapping

* #### Importation des packages

In [1]:
from lxml import html
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib

* #### Obtention de liens des produits

In [2]:
test_url=[]
soup_test = BeautifulSoup(
            urllib.request.urlopen("https://www.auchan.fr/nos-boutiques/ca-b00?page=1"))
for lien in soup_test.find_all('a', href=True): #les url se trouvent dans une balise 'a' et renseignées au niveau d el'attribut href
          if 'pr-' in lien['href']: #on récupère uniquement les url des articles
              test_url.append(lien['href'])



In [3]:
len(test_url)

30

In [4]:
url_pages=["https://www.auchan.fr/nos-boutiques/ca-b00?page=%d" %(page_count) \
             for page_count in range(1 , 381)]


Ce fonction nous permet de suivre le progres du boucle

In [5]:
def printper(current , total):
    unit = total/100
    print("{}% done\n".format(str(int(current//unit))))

In [6]:
url_articles=[]
url_count = 0
url_unit = len(url_pages)/100
url_announced = 0
for page in url_pages : 
    url_count += 1
    soup = BeautifulSoup(urllib.request.urlopen(page))
    if url_count//url_unit != url_announced:
        print("Looking for articles...")
        url_announced = url_count//url_unit
        printper(url_count, len(url_pages))
    for lien in soup.find_all('a', href=True): #les url se trouvent dans une balise 'a' et renseignées au niveau d el'attribut href
             if 'pr-' in lien['href']: #on récupère uniquement les url des articles
                 url_articles.append(lien['href'])
nart = len(url_articles)
print ("{} products found.\n".format(nart ))                     


Looking for articles...
1% done

Looking for articles...
2% done

Looking for articles...
3% done

Looking for articles...
4% done

Looking for articles...
5% done

Looking for articles...
6% done

Looking for articles...
7% done

Looking for articles...
8% done

Looking for articles...
9% done

Looking for articles...
10% done

Looking for articles...
11% done

Looking for articles...
12% done

Looking for articles...
13% done

Looking for articles...
14% done

Looking for articles...
15% done

Looking for articles...
16% done

Looking for articles...
17% done

Looking for articles...
18% done

Looking for articles...
19% done

Looking for articles...
20% done

Looking for articles...
21% done

Looking for articles...
22% done

Looking for articles...
23% done

Looking for articles...
24% done

Looking for articles...
25% done

Looking for articles...
26% done

Looking for articles...
27% done

Looking for articles...
28% done

Looking for articles...
29% done

Looking for articles...

In [7]:
len(url_articles)#nous trouvons bien la totalité des produits car il y a 30 produits par page pour environ 343 pages.

10431

* #### Scrapping

In [25]:
def assigner(elem=[]):
    # This function evaluates the xpath result before adding it to the DB. 
    # Upon an empty result, it replaces it for [""] so that a dataframe can be constructed.
    if elem:
        return elem
    else:
        return [""]

donnes_scrap = {"titres":[] ,  
                "urls":[] , 
                "informations":[],
                "allergenes":[],
               "score":[]}

url="https://www.auchan.fr"

unit = nart/100
current = 0
announced = 0

for article in url_articles:
    current += 1
    page = requests.get(url + article)
    tree = html.fromstring(page.content)
    informations = tree.xpath('//h1[@class="site-breadcrumb__title"]/a[@href]/text()') + \
                    tree.xpath('//div[@class="product-description__content"]/p/text()')

    for i in informations:   
        if "Ingrédients" in i:
            donnes_scrap["informations"].append(i)
            donnes_scrap["titres"].append(informations[0]) 
            donnes_scrap["urls"].append(url+article)
            donnes_scrap["allergenes"].append( \
                assigner(tree.xpath('//span[@class="product-features__value"]/text() \
                    | //div[button/h4[@class="product-description__title"]="Allergènes"]\
                    /div[@class="product-description__content"]/p/text()')))
            donnes_scrap["score"] += \
                assigner(tree.xpath('//div[@class="product-nutriscore"]//img/@alt'))
            
    if current//unit != announced:
        print("Looking for information...")
        announced = current//unit
        printper(current, nart)

In [26]:
for cle in donnes_scrap:
    print("Nb d'objets en {}: {}".format(cle , len(donnes_scrap[cle])))

Nb d'objets en titres: 74
Nb d'objets en urls: 74
Nb d'objets en informations: 74
Nb d'objets en allergenes: 74
Nb d'objets en score: 74


In [27]:
df = pd.DataFrame(donnes_scrap)

In [28]:
len(df)

74

In [29]:
df.to_csv("scrapped.csv", sep=";" , encoding='utf-8')

# Fin Labo